In [1]:
import cv2
import matplotlib.pyplot as plt
import importlib
import os
from ipywidgets import interact
from ipywidgets import interact_manual
from IPython.display import display
from ipywidgets import widgets

In [2]:
import ipynb.fs.full.wspFFA as wspFirefly
import ipynb.fs.full.wspCS as wspCuckooSearch
import ipynb.fs.full.wspKH as wspKrillHerd
import ipynb.fs.full.wspEHO as wspElephantHerding
import ipynb.fs.full.wspABC as wspArtificialBeeColony

import ipynb.fs.full.plotGraph as plotGraph
import ipynb.fs.full.wspMultithreshold as wspMultithreshold

importlib.reload(wspFirefly)
importlib.reload(wspCuckooSearch)
importlib.reload(wspKrillHerd)
importlib.reload(wspElephantHerding)
importlib.reload(wspArtificialBeeColony)

importlib.reload(plotGraph)
importlib.reload(wspMultithreshold)

<module 'ipynb.fs.full.wspMultithreshold' (/mnt/6884ACCB84AC9CD6/OneDrive - FEI/Aulas/Graduacao/8 - Semestre/CC8421 - TRABALHO FINAL DE CURSO II/image-segmentation/wspMultithreshold.ipynb)>

In [3]:
def readImage(img_path):
    image = cv2.imread(img_path)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return img

In [4]:
bio_algorithm = ['FFA','KH', 'CS', 'ABC', 'EHO']

In [5]:
folder_dir = "images"

all_images = []

for file in os.listdir(folder_dir):
    if file.endswith('.png') or file.endswith('.jpg'):
        all_images.append(file)

In [6]:
def main(image, algorithm, dimension, q):
    img = readImage(f"images/{image}")

    plotGraph.plot_image_histogram(img)
    
    hist, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(img, algorithm, dimension, q)

    plotGraph.plot_image_histogram_threshold(img_thres, hist, best_thresholds)

In [7]:
interact_manual(main, image=all_images, algorithm=bio_algorithm, 
    dimension=widgets.IntSlider(min=1, max=5, step=1, value=3),
    q=widgets.FloatSlider(min=-2, max=2, step=0.1, value=0.5))

interactive(children=(Dropdown(description='image', options=('lenna.png', 'ocranio.jpg', 'piano.jpg', 'zcam.pn…

<function __main__.main(image, algorithm, dimension, q)>